# Getting the latent space of pre-trained models and predict task


In [7]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
import importlib
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import optuna
import imaplib


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score


from get_pretrain_encoder import get_pretrain_encoder_from_modelID, get_input_data
from latent_task_predict_pretrain import log_reg_multi_class, ridge_regression_predict


%who



DataLoader	 LabelEncoder	 TensorDataset	 accuracy_score	 f1_score	 get_input_data	 get_pretrain_encoder_from_modelID	 imaplib	 importlib	 
input_data_location	 label_encoder	 log_reg_multi_class	 ml_code_path	 model_id_list	 nn	 np	 optim	 optuna	 
os	 pd	 precision_score	 pretrain_save_dir	 random	 recall_score	 ridge_regression_predict	 roc_auc_score	 task_list_cat	 
task_list_num	 torch	 


In [8]:
import get_pretrain_encoder
importlib.reload(get_pretrain_encoder)
from get_pretrain_encoder import get_pretrain_encoder_from_modelID
# getting the latent space for the model
(encoder, Z_all, Z_train, Z_val, Z_test, y_data_all, y_data_train, y_data_val, y_data_test)=get_pretrain_encoder_from_modelID(model_id, input_data_location, pretrain_save_dir, ml_code_path)



/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:60: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-36657
Deleted existing file at /home/leilapirhaji/pretrained_models/pretrained_models/RCC-36657/encoder_state_dict.pth


Fetching file...: 0 [00:00, ?/s]

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-36657/metadata


/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:173: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder_state_dict = torch.load(model_encoder_file)


In [9]:
print (Z_all.shape, Z_train.shape, Z_val.shape, Z_test.shape, y_data_all.shape, y_data_train.shape, y_data_val.shape, y_data_test.shape)

(20548, 200) (14391, 200) (3074, 200) (3083, 200) (20548, 58) (14391, 58) (3074, 58) (3083, 58)


In [10]:
#def evalute_pretrain_latent_extra_task(model_id, input_data_location, pretrain_save_dir, task_list_cat, task_list_num):
input_data_location='/home/leilapirhaji/PROCESSED_DATA_2'
pretrain_save_dir='/home/leilapirhaji/pretrained_models' 



#tasks to predict using encoder
task_list_cat=[ 'Study ID', 'is Female', 'is Pediatric', 'Cohort Label v0','Smoking Status', 'Cancer Risk' ]

task_list_num=[ 'BMI', 'Age' ]

model_id='RCC-36657'

print (f'model_id is {model_id}')


# getting the latent space for the model
#(encoder, Z_all, Z_train, Z_val, Z_test, y_data_all, y_data_train, y_data_val, y_data_test)=get_pretrain_encoder_from_modelID(model_id, input_data_location, pretrain_save_dir, ml_code_path)




# Now use the latnet space to predict the tasks

model_results = {'Model ID': model_id}

# Predict the categorical tasks

for task in task_list_cat:

    (val_accuracy, test_accuracy) = log_reg_multi_class(task, Z_train, y_data_train, Z_val, y_data_val, Z_test, y_data_test)

    # Store the results in the dictionary
    model_results[f'{task} Val Accuracy'] = val_accuracy
    model_results[f'{task} Test Accuracy'] = test_accuracy

    print(f'{task} Val Accuracy: {val_accuracy:.4f}')
    print(f'{task} Test Accuracy: {test_accuracy:.4f}')


#now evaluting numercal task predictions
for task in task_list_num:

    (val_mse, val_mae, val_r2, test_mse, test_mae, test_r2)= ridge_regression_predict(task, Z_train, y_data_train, Z_val, y_data_val, Z_test, y_data_test)

    # Store the results in the dictionary
    # model_results[f'{task} Val MSE'] = val_mse
    model_results[f'{task} Val MAE'] = val_mae
    # model_results[f'{task} Val R2'] = val_r2
    # model_results[f'{task} Test MSE'] = test_mse
    model_results[f'{task} Test MAE'] = test_mae
    # model_results[f'{task} Test R2'] = test_r2

    print(f'{task} Val MAE : {val_mae:.4f}')
    print(f'{task} Test MAE : {test_mae:.4f}')
    


#return model_results


model_id is RCC-36657
Study ID Val Accuracy: 0.9525
Study ID Test Accuracy: 0.9458
is Female Val Accuracy: 0.8043
is Female Test Accuracy: 0.7567
is Pediatric Val Accuracy: 0.9808
is Pediatric Test Accuracy: 0.9818
Cohort Label v0 Val Accuracy: 0.9619
Cohort Label v0 Test Accuracy: 0.9578
Smoking Status Val Accuracy: 0.9311
Smoking Status Test Accuracy: 0.9314
Cancer Risk Val Accuracy: 0.9233
Cancer Risk Test Accuracy: 0.9190
BMI Val MAE : 3.9232
BMI Test MAE : 3.9238


/home/leilapirhaji/.local/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.93296e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/leilapirhaji/.local/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=5.43836e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Age Val MAE : 9.9546
Age Test MAE : 9.7178


In [11]:
model_results

{'Model ID': 'RCC-36657',
 'Study ID Val Accuracy': 0.9525048796356539,
 'Study ID Test Accuracy': 0.9458319818358741,
 'is Female Val Accuracy': 0.8043478260869565,
 'is Female Test Accuracy': 0.7566666666666667,
 'is Pediatric Val Accuracy': 0.9808067664281067,
 'is Pediatric Test Accuracy': 0.9818358741485566,
 'Cohort Label v0 Val Accuracy': 0.9619388418998048,
 'Cohort Label v0 Test Accuracy': 0.957833279273435,
 'Smoking Status Val Accuracy': 0.9311475409836065,
 'Smoking Status Test Accuracy': 0.9313725490196079,
 'Cancer Risk Val Accuracy': 0.9233067729083665,
 'Cancer Risk Test Accuracy': 0.9190444591904446,
 'BMI Val MAE': 3.9232283557165877,
 'BMI Test MAE': 3.923811241793968,
 'Age Val MAE': 9.95463990170786,
 'Age Test MAE': 9.717826384024695}

In [2]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
import importlib
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


from get_pretrain_encoder import get_pretrain_encoder_from_modelID
from latent_task_predict_pretrain import log_reg_multi_class, ridge_regression_predict




[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.


In [3]:
input_data_location='/home/leilapirhaji/PROCESSED_DATA_2'
pretrain_save_dir='/home/leilapirhaji/pretrained_models' 



#tasks to predict using encoder
task_list_cat=[ 'Study ID', 'is Female', 'is Pediatric', 'Cohort Label v0','Smoking Status', 'Cancer Risk' ]

task_list_num=[ 'BMI', 'Age' ]

model_id='RCC-36657'

model_results=evalute_pretrain_latent_extra_task(model_id, input_data_location, pretrain_save_dir, task_list_cat, task_list_num)

model_id is RCC-36657


/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:60: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)
[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-36657
Deleted existing file at /home/leilapirhaji/pretrained_models/pretrained_models/RCC-36657/encoder_state_dict.pth


Fetching file...: 0 [00:00, ?/s]

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-36657/metadata


/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:173: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder_state_dict = torch.load(model_encoder_file)


TypeError: expected Tensor as element 0 in argument 0, but got DataFrame

## getting the input data

In [8]:
#input data
model_id_list=[]

input_data_location='/home/leilapirhaji/PROCESSED_DATA_2'
pretrain_save_dir='/home/leilapirhaji/pretrained_models' 



#tasks to predict using encoder
task_list_cat=[ 'Study ID', 'is Female', 'is Pediatric', 'Cohort Label v0','Smoking Status', 'Cancer Risk' ]

task_list_num=[ 'BMI', 'Age' ]

(X_data_all, y_data_all, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)= get_input_data(input_data_location)

X_data_all.shape, y_data_all.shape, X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape


/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:159: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)


((20548, 2736),
 (20548, 58),
 (14391, 2736),
 (14391, 58),
 (3074, 2736),
 (3074, 58),
 (3083, 2736),
 (3083, 58))

## Get the encoder and latent space with averaging 

In [36]:
import importlib
import get_pretrain_encoder
importlib.reload(get_pretrain_encoder)
from get_finetune_encoder import get_finetune_encoder_from_modelID

model_id='RCC-36657'
#model_id='RCC-36682' #bad recon loss

(encoder, Z_all, Z_train, Z_val, Z_test, y_data_all, y_data_train, y_data_val, y_data_test)=get_pretrain_encoder_from_modelID(model_id, input_data_location, pretrain_save_dir, ml_code_path)


/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:60: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-36657
Deleted existing file at /home/leilapirhaji/pretrained_models/pretrained_models/RCC-36657/encoder_state_dict.pth


Fetching file...: 0 [00:00, ?/s]

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-36657/metadata


/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:117: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_kwargs = convert_neptune_kwargs(load_kwargs)


In [37]:

print (Z_all.shape, Z_train.shape, Z_val.shape, Z_test.shape, y_data_all.shape, y_data_train.shape, y_data_val.shape, y_data_test.shape)

(20548, 200) (14391, 200) (3074, 200) (3083, 200) (20548, 58) (14391, 58) (3074, 58) (3083, 58)


## visulaizeing latent sapce

## Using latent space to predict categorical tasks

In [38]:
import latent_task_predict_pretrain
importlib.reload(latent_task_predict_pretrain)
from latent_task_predict_pretrain import log_reg_multi_class, ridge_regression_predict

#tasks to predict using encoder
task_list_cat=[ 'Study ID', 'is Female', 'is Pediatric', 'Cohort Label v0','Smoking Status', 'Cancer Risk' ]


task='Cancer Risk'

(best_val_accuracy, test_accuracy) = log_reg_multi_class(task, Z_train, y_data_train, Z_val, y_data_val, Z_test, y_data_test)

best_val_accuracy, test_accuracy


(0.9233067729083665, 0.9190444591904446)

## Using latent space to predict numerical tasks

In [39]:
Z_test

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
F088_210813_M462_115.mzML,0.016151,-0.036399,-0.021051,-0.004889,0.325092,-0.019084,-0.017611,0.031336,0.005833,-0.012649,...,-0.014541,0.005112,0.032623,0.022529,0.033487,-0.015567,0.018097,0.035755,0.026799,-0.025056
F088_210524_M462_145.mzML,-0.050986,-0.024688,-0.015108,0.012049,0.573699,-0.007681,-0.032105,0.003416,0.090084,-0.044102,...,0.074447,-0.046749,-0.050500,0.010690,0.042673,-0.080158,-0.028781,0.077156,-0.045460,-0.000707
F088_210730_M462_159.mzML,-0.040502,-0.011032,0.034642,0.040441,0.867057,0.019429,-0.017247,0.025385,-0.031466,0.007573,...,0.012122,-0.020547,-0.020013,0.001909,0.030952,-0.025218,0.004636,0.030134,0.032337,-0.015429
F088_210726_M462_031.mzML,-0.005879,-0.043891,0.001956,-0.028704,0.576279,-0.004865,0.032372,0.022575,0.033603,0.008344,...,-0.036182,0.019366,0.039915,0.039618,0.041128,-0.022309,-0.039295,0.024300,0.040656,-0.002575
F088_210806_M462_215.mzML,-0.025132,0.007773,0.013963,0.026506,0.730099,0.028892,0.001381,-0.009753,-0.015385,0.029366,...,-0.069029,0.036645,0.009928,0.023202,0.019965,0.020323,-0.027526,0.027571,0.021643,-0.000180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22569A_1x_Batch_20_A.mzML,-0.010247,0.010016,0.006770,-0.022723,0.017137,0.000513,0.004782,-0.005852,0.001723,0.009563,...,-0.022247,0.019384,0.013882,0.028416,-0.002800,0.007385,-0.024806,0.018897,0.005400,0.010338
20738N_4x_Batch_16_A.mzML,-0.022120,-0.009530,0.010476,-0.003063,0.049409,-0.010626,-0.028540,-0.006562,0.003547,-0.019988,...,-0.018364,-0.021785,-0.008693,0.021442,-0.014655,0.008942,-0.000733,0.002410,0.015105,0.001419
24931B_2x_Batch_12_A.mzML,-0.014724,0.003587,0.015515,-0.012570,0.096546,-0.002493,-0.017939,-0.011354,-0.000227,-0.006521,...,-0.011726,-0.006589,-0.002971,0.006746,-0.014851,0.006550,-0.003540,0.013042,0.010477,0.003947
20900W_4x_Batch_20_A.mzML,-0.013594,0.010420,0.016049,-0.021255,-0.177868,0.004597,-0.009599,-0.001721,0.006687,0.013425,...,-0.027656,0.009300,0.003024,0.038464,-0.018237,0.008171,-0.017060,0.008208,-0.014782,0.009877


In [50]:
import latent_task_predict_pretrain
importlib.reload(latent_task_predict_pretrain)
from latent_task_predict_pretrain import log_reg_multi_class, ridge_regression_predict

task_list_num=[ 'BMI', 'Age' ]
task='BMI'

(best_val_mse, val_mae, val_r2, test_mse, test_mae, test_r2)= ridge_regression_predict(task, Z_train, y_data_train, Z_val, y_data_val, Z_test, y_data_test)

best_val_mse, val_mae, val_r2, test_mse, test_mae, test_r2

/home/leilapirhaji/.local/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.93355e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


(31.474254022919432,
 3.923244522162404,
 0.16234745248223503,
 33.136116736617716,
 3.9238094311236593,
 0.21686598261764212)

In [12]:
import eval_pretrain_latent
importlib.reload(eval_pretrain_latent)
from eval_pretrain_latent import evalute_pretrain_latent_extra_task

model_results = evalute_pretrain_latent_extra_task(model_id, input_data_location, pretrain_save_dir, task_list_cat, task_list_num)

model_results


model_id is RCC-36657


/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:60: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-36657
Deleted existing file at /home/leilapirhaji/pretrained_models/pretrained_models/RCC-36657/encoder_state_dict.pth


Fetching file...: 0 [00:00, ?/s]

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-36657/metadata


/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:173: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder_state_dict = torch.load(model_encoder_file)


(20548, 200) (14391, 200) (3074, 200) (3083, 200) (20548, 58) (14391, 58) (3074, 58) (3083, 58)
Study ID Val Accuracy: 0.9525
Study ID Test Accuracy: 0.9458
is Female Val Accuracy: 0.8043
is Female Test Accuracy: 0.7567
is Pediatric Val Accuracy: 0.9808
is Pediatric Test Accuracy: 0.9818
Cohort Label v0 Val Accuracy: 0.9619
Cohort Label v0 Test Accuracy: 0.9578
Smoking Status Val Accuracy: 0.9311
Smoking Status Test Accuracy: 0.9314
Cancer Risk Val Accuracy: 0.9233
Cancer Risk Test Accuracy: 0.9190


/home/leilapirhaji/.local/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.93296e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/leilapirhaji/.local/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=5.43836e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


BMI Val MAE : 3.9232
BMI Test MAE : 3.9238
Age Val MAE : 9.9546
Age Test MAE : 9.7178


{'Model ID': 'RCC-36657',
 'Study ID Val Accuracy': 0.9525048796356539,
 'Study ID Test Accuracy': 0.9458319818358741,
 'is Female Val Accuracy': 0.8043478260869565,
 'is Female Test Accuracy': 0.7566666666666667,
 'is Pediatric Val Accuracy': 0.9808067664281067,
 'is Pediatric Test Accuracy': 0.9818358741485566,
 'Cohort Label v0 Val Accuracy': 0.9619388418998048,
 'Cohort Label v0 Test Accuracy': 0.957833279273435,
 'Smoking Status Val Accuracy': 0.9311475409836065,
 'Smoking Status Test Accuracy': 0.9313725490196079,
 'Cancer Risk Val Accuracy': 0.9233067729083665,
 'Cancer Risk Test Accuracy': 0.9190444591904446,
 'BMI Val MAE': 3.9232283557165877,
 'BMI Test MAE': 3.923811241793968,
 'Age Val MAE': 9.95463990170786,
 'Age Test MAE': 9.717826384024695}

In [13]:
model_results

{'Model ID': 'RCC-36657',
 'Study ID Val Accuracy': 0.9525048796356539,
 'Study ID Test Accuracy': 0.9458319818358741,
 'is Female Val Accuracy': 0.8043478260869565,
 'is Female Test Accuracy': 0.7566666666666667,
 'is Pediatric Val Accuracy': 0.9808067664281067,
 'is Pediatric Test Accuracy': 0.9818358741485566,
 'Cohort Label v0 Val Accuracy': 0.9619388418998048,
 'Cohort Label v0 Test Accuracy': 0.957833279273435,
 'Smoking Status Val Accuracy': 0.9311475409836065,
 'Smoking Status Test Accuracy': 0.9313725490196079,
 'Cancer Risk Val Accuracy': 0.9233067729083665,
 'Cancer Risk Test Accuracy': 0.9190444591904446,
 'BMI Val MAE': 3.9232283557165877,
 'BMI Test MAE': 3.923811241793968,
 'Age Val MAE': 9.95463990170786,
 'Age Test MAE': 9.717826384024695}